## Description

This notebook performs the experiments for nudge impacts for distributions generated by Ising models.

In [2]:
import time
import os
import numpy as np
import networkx as nx
import pandas as pd
from r0bit import ising
import ising_model

### Generate the network used for the Ising model

#### Network settings

In [3]:
number_of_nodes = 100
network_degree = 2

#### Generate the network

In [4]:
network_backup = ising.generate_powerlaw_network(number_of_nodes, network_degree)
network = network_backup.copy()
ising_model.set_values_nodes_uniform(network)

### Gather samples to estimate the distributions by simulating the ising model

#### Ising model settings

In [5]:
temperature = 2.5

#### simulation settings

In [6]:
timesteps = 50
number_of_samples = 100
number_of_distributions = 30

#### perform the simulation and gather the samples and create the distributions

In [7]:
start = time.time()

samples_folder = "ising_samples2"
if not os.path.exists(samples_folder):
    os.makedirs(samples_folder)
    
for distribution_number in range(number_of_distributions):
    print("distribution number {}".format(distribution_number))
    network = ising.generate_powerlaw_network(number_of_nodes, network_degree)
    ising_model.set_values_nodes_uniform(network)
    nx.write_gpickle(network, "{}/network{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, distribution_number, number_of_nodes, network_degree, temperature
    ))
    samples = pd.DataFrame(
        data=np.zeros((number_of_samples, len(network.nodes()))), 
        index=range(number_of_samples), 
        columns=network.nodes()
    )
    for sample_number in range(number_of_samples):
        if sample_number%100 == 0 and sample_number != 0:
            print("sample number {}".format(sample_number))
        
        ising_model.update_network(network, timesteps, temperature, update_method="glauber")
        for node in network.nodes():
            samples.at[sample_number, node] = network.node[node]["value"]
            
    samples.to_pickle("{}/samples{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, distribution_number, number_of_nodes, network_degree, temperature
    ))
    #print(samples.loc[:, [1, 2]])
    
print("run time {}".format(time.time()-start))

distribution number 0
distribution number 1
distribution number 2
distribution number 3
distribution number 4
distribution number 5
distribution number 6
distribution number 7
distribution number 8
distribution number 9
distribution number 10
distribution number 11
distribution number 12
distribution number 13
distribution number 14
distribution number 15
distribution number 16
distribution number 17
distribution number 18
distribution number 19
distribution number 20
distribution number 21
distribution number 22
distribution number 23
distribution number 24
distribution number 25
distribution number 26
distribution number 27
distribution number 28
distribution number 29
run time 805.256852865


### Convert the samples to distributions

#### settings for creating the distributions

In [8]:
number_of_states = 2
max_number_of_neighbors = 9
distributions_folder = "ising_distributions"
if not os.path.exists(distributions_folder):
    os.makedirs(distributions_folder)

samples_folder = "ising_samples1"

In [9]:

for count in range(number_of_distributions):
    network = nx.read_gpickle("{}/network{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, count, number_of_nodes, network_degree, temperature
    ))
    samples = pd.read_pickle("{}/samples{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, count, number_of_nodes, network_degree, temperature
    ))
    selected_nodes = ising_model.select_nodes(network, max_number_of_neighbors)
    for selected_node in selected_nodes:
        print(list(network.neighbors(selected_node)))
        distribution = ising_model.convert_samples_to_dist(
            samples, selected_node, list(network.neighbors(selected_node)), number_of_states
        )
        print(distribution.shape)
        output_label = set([len(distribution.shape) - 1])
        input_labels = set(range(len(distribution.shape) - 1))
        marginal, conditional = ising_model.produce_marginal_and_conditional(
            distribution, input_labels, output_label
        )

        number_of_neighbors = len(list(network.neighbors(selected_node))) 
        filename_marginal = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        filename_conditional = "{}/conditional{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        np.save(filename_marginal, marginal)
        np.save(filename_conditional, conditional)
        #print(distribution)
        
        #input_distribution = 
        #conditional_output_distribution = 
        #save distributions



[16]
(2, 2)
[26, 12]
(2, 2, 2)
[19, 12, 23]
(2, 2, 2, 2)
[52, 59, 12, 31]
(2, 2, 2, 2, 2)
[30, 19, 12, 13, 70]
(2, 2, 2, 2, 2, 2)
[96, 35, 4, 12, 19, 21]
(2, 2, 2, 2, 2, 2, 2)
[96, 70, 42, 12, 52, 23, 30]
(2, 2, 2, 2, 2, 2, 2, 2)
[35, 12, 76, 46, 19, 52, 21, 31]
(2, 2, 2, 2, 2, 2, 2, 2, 2)
[92]
(2, 2)
[44, 77]
(2, 2, 2)
[40, 44, 46]
(2, 2, 2, 2)
[89, 42, 12, 62]
(2, 2, 2, 2, 2)
[31, 42, 27, 28, 39]
(2, 2, 2, 2, 2, 2)
[74, 76, 13, 45, 22, 62]
(2, 2, 2, 2, 2, 2, 2)
[65, 66, 42, 44, 53, 55, 24, 58]
(2, 2, 2, 2, 2, 2, 2, 2, 2)
[64]
(2, 2)
[64, 34]
(2, 2, 2)
[64, 57, 52]
(2, 2, 2, 2)
[64, 1, 99, 53, 57]
(2, 2, 2, 2, 2, 2)
[64, 97, 39, 76, 57, 30, 31]
(2, 2, 2, 2, 2, 2, 2, 2)
[64, 99, 42, 76, 52, 20, 93, 95]
(2, 2, 2, 2, 2, 2, 2, 2, 2)
[18]
(2, 2)
[18, 50]
(2, 2, 2)
[64, 44, 93]
(2, 2, 2, 2)
[64, 50, 19, 93]
(2, 2, 2, 2, 2)
[80, 64, 2, 19, 93]
(2, 2, 2, 2, 2, 2)
[2, 68, 50, 90, 28, 94]
(2, 2, 2, 2, 2, 2, 2)
[64, 34, 6, 7, 48, 50, 58]
(2, 2, 2, 2, 2, 2, 2, 2)
[64, 2, 4, 11, 50, 53, 88, 68]
(2

(2, 2)
[8, 43]
(2, 2, 2)
[8, 24, 3]
(2, 2, 2, 2)
[31, 33, 43, 55]
(2, 2, 2, 2, 2)
[8, 98, 83, 20, 55]
(2, 2, 2, 2, 2, 2)
[0, 5, 8, 77, 16, 55]
(2, 2, 2, 2, 2, 2, 2)
[67, 8, 43, 12, 11, 55, 62]
(2, 2, 2, 2, 2, 2, 2, 2)
[66, 5, 8, 45, 79, 53, 23, 60]
(2, 2, 2, 2, 2, 2, 2, 2, 2)
[33, 5, 8, 43, 45, 84, 21, 55, 26]
(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)


### Perform nudge experiments

In [10]:

for count in range(number_of_distributions):
    for number_of_neighbors in range(1, max_number_of_neighbors):
        filename_marginal = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        filename_conditional = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        try:
            marginal = np.load(filename_marginal)
            conditional = np.load(filename_conditional)            
        except IOError:
            print("file not found")

file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found


### Inspect entropy of a certain node

Inspect the entropy of a certain node in the network for a certain degree

In [11]:
import collections

def get_samples_nodes(network, selected_nodes, temperature):
    samples_selected_nodes = {selected_node:[] for selected_node in selected_nodes}

    for timestep in range(number_of_timesteps):
        for selected_node in selected_nodes:
            samples_selected_nodes[selected_node].append(network.node[selected_node]["value"])

        ising_model.update_network(network, 1, temperature)
        
    return samples_selected_nodes
    
def calculate_flip_probability(series):
    changes = [0 if series[i]==series[i+1] else 1 for i in range(len(series)-1)]
    #print(changes)
    #print(collections.Counter(changes))
    return collections.Counter(changes)[1] / float(len(series))

def get_flip_probabilities(network, selected_nodes, temperature):
    samples_selected_nodes = get_samples_nodes(network, selected_nodes, temperature)
    number_of_neighbors_to_flip_probs = {}
    for selected_node in selected_nodes:
        #print(len(samples_selected_nodes[selected_node]))
        flip_probability = calculate_flip_probability(samples_selected_nodes[selected_node])
#         print("number of neighbors {}, flip probability {}".format(
#             len(list(network.neighbors(selected_node))), flip_probability
#         ))
        number_of_neighbors_to_flip_probs[len(list(network.neighbors(selected_node)))] = flip_probability
        
    return number_of_neighbors_to_flip_probs
        

In [12]:
temperature = 2.5
number_of_timesteps = 100
max_number_of_neighbors = 8

flip_probabilities = {i:[] for i in range(1, max_number_of_neighbors+1, 1)}
for timestep in range(number_of_timesteps):
    network = ising.generate_powerlaw_network(number_of_nodes, network_degree)
    ising_model.set_values_nodes_uniform(network)
    selected_nodes = ising_model.select_nodes(network, max_number_of_neighbors)

    number_of_neighbors_to_flip_probs = get_flip_probabilities(network, selected_nodes, temperature)
    for number_of_neighbors, flip_probs in number_of_neighbors_to_flip_probs.items():
        flip_probabilities[number_of_neighbors].append(flip_probs)


In [13]:
for i in range(1, max_number_of_neighbors+1, 1):
    print(np.mean(flip_probabilities[i]))

0.2995
0.2786
0.2679
0.229285714286
0.205833333333
0.195
0.163793103448
0.143287671233
